In [ ]:
import pandas as pd
import numpy as np
import datetime
import tensorflow as tf
import io
from google.colab import files
from keras.callbacks import LearningRateScheduler
from matplotlib import pyplot
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from IPython.display import Audio

#print("Versão do TensorFlow:",tf.__version__)

In [ ]:
def step_decay_schedule(initial_lr, decay_factor, step_size):
    '''
    Wrapper function to create a LearningRateScheduler with step decay schedule.
    '''
    def schedule(epoch):
        return initial_lr * (decay_factor ** np.floor(epoch/step_size))
    
    return LearningRateScheduler(schedule)

# This function keeps the initial learning rate for the first ten epochs
# and decreases it exponentially after that.
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

# Function for decaying the learning rate.
# You can define any decay function you need.
def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

**1. FUNÇÕES**

In [ ]:
def funcMontante(montante):
  


# Coleta os dados totais dos times a cada round
def getTotalPreviousRounds(currentround,menorDatabase):

    dic = {}

    if currentround >= 14:
        trainDataMax = (currentround*10)-10 # coleta os dados da linha 0 até a última linha antes da round atual;
        for i in range(40,trainDataMax,1):
              team = str(menorDatabase.iloc[i,0])
              if(team not in dic):
                  dic[team] = {'GamesH':0,'GamesA':0,'FTHG':0,'FTAG':0,'HS':0,'AS':0,'HST':0,'AST':0}  # Define as stats base zero para o novo time;
                  
                  dic[team]['GamesH'] = dic[team]['GamesH'] + 1 #|------------------------> Games: quantidade de jogos como mandante;
                  dic[team]['FTHG'] = dic[team]['FTHG'] + int(menorDatabase.iloc[i,2]) #|--------> FTHG: Goals do Mandante no jogo todo; 
                  dic[team]['HS'] = dic[team]['HS'] + int(menorDatabase.iloc[i,5]) #|------------> HS: Chutes ao gol do Mandante;
                  dic[team]['HST'] = dic[team]['HST'] + int(menorDatabase.iloc[i,7]) #|----------> HST: Chutes no alvo do Mandante;

              else:
                  dic[team]['GamesH'] = dic[team]['GamesH'] + 1 #|------------------------> Games: quantidade de jogos como mandante;
                  dic[team]['FTHG'] = dic[team]['FTHG'] + int(menorDatabase.iloc[i,2]) #|--------> FTHG: Goals do Mandante no jogo todo;  
                  dic[team]['HS'] = dic[team]['HS'] + int(menorDatabase.iloc[i,5]) #|------------> HS: CHutes ao gol do Mandante;
                  dic[team]['HST'] = dic[team]['HST'] + int(menorDatabase.iloc[i,7]) #|----------> ST: Chutes no alvo do Mandante;

        for i in range(trainDataMax):
              team = str(menor.iloc[i,1])
              if(team not in dic):
                  dic[team] = {'GamesH':0,'GamesA':0,'FTHG':0,'FTAG':0,'HS':0,'AS':0,'HST':0,'AST':0}  # Repetindo para o caso do time ainda não ter sido inserido na lista;
              dic[team]['GamesA'] = dic[team]['GamesA'] + 1 
              dic[team]['FTAG'] = dic[team]['FTAG'] + int(menorDatabase.iloc[i,3]) #|-----------> FTAG: Goals do Visitante no jogo todo;  
              dic[team]['AS'] = dic[team]['AS'] + int(menorDatabase.iloc[i,6]) #|---------------> AS: Chutes ao gol do Visitante;
              dic[team]['AST'] = dic[team]['AST'] + int(menorDatabase.iloc[i,8]) #|-------------> AST: Chutes no alvo do Visitante;
    
    elif currentround == 13:
          trainDataMax = 40 # coleta os dados da linha 0 até a linha 40
          for i in range(0,trainDataMax,1):
              team = str(menorDatabase.iloc[i,0])
              if(team not in dic):
                  dic[team] = {'GamesH':0,'GamesA':0,'FTHG':0,'FTAG':0,'HS':0,'AS':0,'HST':0,'AST':0}  # Define as stats base zero para o novo time;
                  
                  dic[team]['GamesH'] = dic[team]['GamesH'] + 1 #|------------------------> Games: quantidade de jogos como mandante;
                  dic[team]['FTHG'] = dic[team]['FTHG'] + int(menorDatabase.iloc[i,2]) #|--------> FTHG: Goals do Mandante no jogo todo; 
                  dic[team]['HS'] = dic[team]['HS'] + int(menorDatabase.iloc[i,5]) #|------------> HS: Chutes ao gol do Mandante;
                  dic[team]['HST'] = dic[team]['HST'] + int(menorDatabase.iloc[i,7]) #|----------> HST: Chutes no alvo do Mandante;

              else:
                  dic[team]['GamesH'] = dic[team]['GamesH'] + 1 #|------------------------> Games: quantidade de jogos como mandante;
                  dic[team]['FTHG'] = dic[team]['FTHG'] + int(menorDatabase.iloc[i,2]) #|--------> FTHG: Goals do Mandante no jogo todo;  
                  dic[team]['HS'] = dic[team]['HS'] + int(menorDatabase.iloc[i,5]) #|------------> HS: CHutes ao gol do Mandante;
                  dic[team]['HST'] = dic[team]['HST'] + int(menorDatabase.iloc[i,7]) #|----------> ST: Chutes no alvo do Mandante;

          for i in range(trainDataMax):
              team = str(menor.iloc[i,1])
              if(team not in dic):
                  dic[team] = {'GamesH':0,'GamesA':0,'FTHG':0,'FTAG':0,'HS':0,'AS':0,'HST':0,'AST':0}  # Repetindo para o caso do time ainda não ter sido inserido na lista;
              dic[team]['GamesA'] = dic[team]['GamesA'] + 1 
              dic[team]['FTAG'] = dic[team]['FTAG'] + int(menorDatabase.iloc[i,3]) #|-----------> FTAG: Goals do Visitante no jogo todo;  
              dic[team]['AS'] = dic[team]['AS'] + int(menorDatabase.iloc[i,6]) #|---------------> AS: Chutes ao gol do Visitante;
              dic[team]['AST'] = dic[team]['AST'] + int(menorDatabase.iloc[i,8]) #|-------------> AST: Chutes no alvo do Visitante;
              
    return dic

def organizeTotals(totalPreviousrounds):
    totalsOrganized = {}
    for elm in totalPreviousrounds.keys():
        totalsOrganized[elm]= {}
    for key,value in zip(totalPreviousrounds.keys(),totalPreviousrounds.values()):
        tempList= []
        for elm in value.keys():
            if(str(elm) == 'GamesH'):   #0
                tempList.append(value[elm])
            if(str(elm) == 'GamesA'):   #1
                tempList.append(value[elm])
            if(str(elm) == 'FTHG'):     #2
                tempList.append(value[elm])
            if(str(elm) == 'FTAG'):     #3
                tempList.append(value[elm])
            if(str(elm) == 'HS'):       #4
                tempList.append(value[elm])
            if(str(elm) == 'AS'):       #5
                tempList.append(value[elm])
            if(str(elm) == 'HST'):      #6
                tempList.append(value[elm])
            if(str(elm) == 'AST'):      #7
                tempList.append(value[elm])
        totalsOrganized[key] = tempList #|--------------------------------> No novo dicionário, o time recebe uma lista 
                                                    # com os dados especificos [gamesH,gamesA,fthg,ftag,hs,as,...];
    return totalsOrganized

def getStatistics(totalsOrganized):
    databaseS = []
    for elm,key in zip(totalsOrganized.values(),totalsOrganized.keys()):
        tempList = [str(key)]                                          # Iniciando a lista temporaria, ja com o nome de cada time;
    
        tempList.append(elm[2]/elm[0])                 # FTHG/jogosH - Média de gol full time/H
        tempList.append(elm[3]/elm[1])                 # FTAG/jogosA -  Média de gols ful time/A
        tempList.append(elm[4]/elm[0])                 # HS/jogosH - Média de Chutes/H
        tempList.append(elm[5]/elm[1])                 # AS/jogosA - Média de Chute/A
        tempList.append(elm[6]/elm[0])                 # HST/jogosH - Média de chutes ao alvo/H
        tempList.append(elm[7]/elm[1])                 # AST/jogosA - Média de chutes ao alvo/A
        databaseS.append(tempList)
    return databaseS  

In [ ]:
def createX_Train(currentRound,x_train_dataset,menorDatabase):
    final_stats = []

    if currentRound == 13:
        previous_rounds_totals = getTotalPreviousRounds(currentRound, menorDatabase)
        totals_organized = organizeTotals(previous_rounds_totals)
        teams_means = getStatistics(totals_organized)

        stats_temp_Home = []
        stats_temp_Away = []

        for i in range(40, 120):

            for j in range(20):

                team_Home = menorDatabase.iloc[i, 0]  # Time mandante
                team_Away = menorDatabase.iloc[i, 1]  # Time visitante

                if team_Home == teams_means[j][0]:
                    temp_list_01 = [teams_means[j][0], teams_means[j][1], teams_means[j][3], teams_means[j][5]]

                    # print(temp_list_01)
                    stats_temp_Home.append(temp_list_01)

                if team_Away == teams_means[j][0]:
                    temp_list_02 = [teams_means[j][0], teams_means[j][2], teams_means[j][4], teams_means[j][6]]
                    stats_temp_Away.append(temp_list_02)

        for i in range(80):
            final_stats.append(
                [stats_temp_Home[i][1], stats_temp_Away[i][1], stats_temp_Home[i][2], stats_temp_Away[i][2],
                 stats_temp_Home[i][3], stats_temp_Away[i][3]])

        for i in range(80):
            x_train_dataset.loc[i] = final_stats[i]
        
    elif currentRound >= 14:

        trainDataMax = (currentRound * 10) - 10
        previous_rounds_totals = getTotalPreviousRounds(currentRound, menorDatabase)
        totals_organized = organizeTotals(previous_rounds_totals)
        teams_means = getStatistics(totals_organized)

        stats_temp_Home = []
        stats_temp_Away = []

        for i in range(40, trainDataMax, 1):
            for j in range(20):

                team_Home = menorDatabase.iloc[i, 0]  # Time mandante
                team_Away = menorDatabase.iloc[i, 1]  # Time visitante

                if team_Home == teams_means[j][0]:
                    temp_list_01 = [teams_means[j][0], teams_means[j][1], teams_means[j][3], teams_means[j][5]]

                    # print(temp_list_01)
                    stats_temp_Home.append(temp_list_01)

                if team_Away == teams_means[j][0]:
                    temp_list_02 = [teams_means[j][0], teams_means[j][2], teams_means[j][4], teams_means[j][6]]
                    stats_temp_Away.append(temp_list_02)

        for i in range(trainDataMax - 40):
            final_stats.append(
                [stats_temp_Home[i][1], stats_temp_Away[i][1], stats_temp_Home[i][2], stats_temp_Away[i][2],stats_temp_Home[i][3], stats_temp_Away[i][3]])

       # x_train_dataset = pd.DataFrame(columns=['FTHGMean', 'FTAGMean', 'HSMean', 'ASMean', 'HSTMean', 'ASTMean'])

        for i in range(trainDataMax - 40):
           x_train_dataset.loc[i] = final_stats[i]
  
    return x_train_dataset

def createY_Train(round_game,menorDatabase):
      if (round_game >= 13 and round_game <= 38):
        actualround = (round_game * 10)-10 # na planilha, a round 9 representa as linhas 80 a 89,por exemplo;
        newDF = menorDatabase.iloc[40:actualround,4]
      elif round_game == 13:
        newDF = menorDatabase.iloc[0:40,4]
      return newDF

# Cria um dataframe de teste X com as estatísticas das 10 partidas da round especificada, a partir do DF de Estatísticas ("gamesS"):
def createX_Test(round_game, menorDatabase, roundGames, roundGames02):
   
   # Organizando as médias dos times por rounds:
  actualround = (round_game*10)-10 # na planilha, a round 9 representa as linhas 80 a 89,por exemplo;
  tempDF = menorDatabase.iloc[actualround:actualround+10,0:]
  mediasHome = roundGames[round_game]    #Coletando as médias dos times Mandantes da round;
  mediasAway = roundGames02[round_game]  #Coletando as médias dos times Visitantes da round;

  listHome = []
  for i in range(10):
    home = tempDF.iloc[i][0]
    for j in range(10):
      if mediasHome[j][0] == home:
        listHome.append([mediasHome[j][1],mediasHome[j][3],mediasHome[j][5]])  #Adicionando uma lista com os valores das tres colunas de Home

  listAway = []
  for i in range(10):
    away = tempDF.iloc[i][1]
    for j in range(10):
      if mediasAway[j][0] == away:
        listAway.append([mediasAway[j][2],mediasAway[j][4],mediasAway[j][6]])  #Adicionando uma lista com os valores das tres colunas de Away

  newDF_02 = pd.DataFrame(columns = ['FTHGMean','FTAGMean','HSMean','ASMean','HSTMean','ASTMean'])
  
  generalList = []
  for i in range(10):
      generalList.append([listHome[i][0],listAway[i][0],listHome[i][1],listAway[i][1],listHome[i][2],listAway[i][2]]) # Juntando os dados de Home e Away, por partida;
      newDF_02.loc[i] = generalList[i] # adicionando cada linha com os valores da generalList;
  
  return newDF_02

# Cria um dataframe de teste Y com os resultados (FTR) em formato binário (vitoria=1 =| derrota/empate=0) das 10 partidas da round especificada, a partir do DF de Resultados (gamesR):
def createY_Test(round_game,menorDatabase):
      if (round_game >= 13 and round_game <= 38):
        actualround = (round_game*10)-10 # na planilha, a round 9 representa as linhas 80 a 89,por exemplo;
        newDF = menorDatabase.iloc[actualround:actualround+10,4]
        return newDF

# Função que retorna um dataframe com os valores Resultado, B365H e OddLayH das 10 partidas da round especificada:
def createBasePredict(round_game,data):
      if (round_game >= 9 and round_game <= 38):
         actualround = (round_game*10)-10 # na planilha, a round 9 representa as linhas 80 a 89,por exemplo;             
         newData = data.drop(columns=['FTHG','FTAG','HS','AS','HST','AST'])
         newDF = newData.iloc[actualround:actualround+10,0:]
         return newDF


**2. MAIN**

**2.1 CARREGANDO O ARQUIVO .CSV**


In [ ]:
# Importando o arquivo e gerando o DataFrame com Pandas;
lista = ['PremierLeague_2010_2011.csv','PremierLeague_2011_2012.csv','PremierLeague_2012_2013.csv','PremierLeague_2013_2014.csv',
         'PremierLeague_2014_2015.csv','PremierLeague_2015_2016.csv','PremierLeague_2016_2017.csv','PremierLeague_2017_2018.csv',
         'PremierLeague_2018_2019.csv','PremierLeague_2019_2020.csv']

dataset = pd.read_csv(lista[9])  # ----> Troque a temporada aqui!!
df = pd.DataFrame(dataset)

**2.2   FILTRANDO DADOS E CRIANDO O DATAFRAME BASE "MENOR" (EXECUTAR UMA ÚNICA VEZ!)**

In [ ]:
#Removendo coluna 'Time'(Tempo, em Inglês): (Atualizado em: 24_02_2021)
lista_colunas = df.columns.values.tolist()
if lista_colunas[2] == 'Time':
  df = df.drop(['Time'],axis=1)

#Corrigindo diferença de quantidade de colunas: (Atualizado em: 24_02_2021)
columns_size = df.shape[1]
if columns_size < 105 and columns_size > 26:
  diff = 105 - columns_size
  i = 0 
  name_column = ''
  while i < diff:
    df[name_column] = ""
    name_column = name_column+"-"
    i = i + 1

# Filtra o DataFrame original com apenas as colunas necessárias para análise;
odds = np.arange(26,105) # atribuindo o intervalo das colunas 25 a 70, onde estão as odds não analisadas. (retorna valores espaçados igualmente dentro de um intervalo definido)
menor = df.drop(df.columns[odds],axis=1,inplace = True)  # Removendo as colunas de odds (permanentemente do DF original)
menor = df.drop(columns = ['Div','Date','HTHG','HTAG','HTR','HF','AF','HC',
                           'AC','HY','AY','HR','AR','Referee','B365D','B365A']) # Adicionei à remoção: B365D,B365A e as colunas dos times(Atualizado em: 25_02_2021)

# Adicionando a coluna oddLayH no DF "menor";
oddsLayH = []
for i in range(380):
  oddH = float(menor.iloc[i][9])
  oddLayH = 1/(1-(1/oddH))
  oddsLayH.append(oddLayH)

menor['oddLayH'] = oddsLayH

# Convertendo resultados de FTR para binario(Vitoria=1, Derrota=0 ,Empate=0) no DF "menor";
listR = []
for i in range(380): 
  if menor.iloc[i,4] == 'H':
    listR.append(1)
  else:
    listR.append(0)
menor['FTR'] = listR

**2.3 CRIANDO AS ESTRUTURAS COM AS MÉDIAS DO MANDANTE E DO VISITANTE**

In [ ]:
# Gerando os dados do time Mandante:
currentround = 13
roundGames = {}
while(currentround <= 38):
    games = []
    actualround = (currentround*10)-10 # na planilha, a round 9 representa as linhas 80 a 89,exemplo;
    for i in range(actualround,actualround+10,1):
        team = menor.iloc[i,0]
        totaisPrevios = getTotalPreviousRounds(currentround,menor)
        totaisOrganizados = organizeTotals(totaisPrevios)
        dataBaseStatistics = getStatistics(totaisOrganizados)
        
        tempList = []   # Gera uma Lista com a info do time mandante, em cada round;
        for elm in dataBaseStatistics:
            if team == elm[0]:
                tempList = elm
        games.append(tempList)
    
    roundGames[currentround] = games

    currentround = currentround + 1;

'''-------------------------------------------------------------------------------------------'''
# Gerando os dados do time Visitante:
currentround = 13
roundGames02 = {}
while(currentround <= 38):
    games = []
    actualround = (currentround*10)-10 # na planilha, a round 9 representa as linhas 80 a 89,exemplo;
    for i in range(actualround,actualround+10,1):
        team = menor.iloc[i,1]
        totaisPrevios = getTotalPreviousRounds(currentround,menor)
        totaisOrganizados = organizeTotals(totaisPrevios)
        dataBaseStatistics = getStatistics(totaisOrganizados)
        
        tempList = []   # Gera uma Lista com a info do time mandante, em cada round;
        for elm in dataBaseStatistics:
            if team == elm[0]:
                tempList = elm
        games.append(tempList)
    
    roundGames02[currentround] = games

    currentround = currentround + 1;

**2.4 EXECUTANDO A REDE NEURAL SIGMÓIDE**

In [ ]:
gamesS = menor.drop(columns = ['B365H','oddLayH','FTR','HomeTeam','AwayTeam']) # DF apenas com as colunas de estatisticas, sem as odds e sem os resultados de todas as partidas;
#gamesR = menor.drop(columns = ['HomeTeam','AwayTeam','FTHG','FTAG','HS','AS','HST','AST','B365H','oddLayH']) # DF apenas com a coluna de resultados (FTR) de todas as partidas;

round_game = 13      # |-------------------------------------------------------------> round inicial para predições;
x_train_dataset = pd.DataFrame(columns=['FTHGMean', 'FTAGMean', 'HSMean', 'ASMean', 'HSTMean', 'ASTMean'])
somaAcuracias = 0

montante = 10000
contadorEmprestimos = 0

errosTotal = 0
acertosTotal = 0
ganhos = []
perdas = []

while round_game <= 38: # |------------------------------------> O tamanho da amostra foi reduzida. Agora só tem 350 linhas para treino e teste;

  if round_game == 13:
      model = tf.keras.models.Sequential()
      model.add(tf.keras.layers.Dense(24, input_dim=6, kernel_initializer='he_uniform', activation='relu'))
      model.add(tf.keras.layers.Dense(units = 24, activation='relu'))
      model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
      model.compile(tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
      mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', verbose = 0, save_best_only=True)  # Vai salvar o primeiro melhor modelo durante as épocas;
  else:
      saved_model = load_model('best_model.h5')
      saved_model.compile(tf.keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])
      mc = ModelCheckpoint('best_model.h5', monitor = 'val_loss', mode='min', verbose = 0, save_best_only=True) # Vai salvar novo melhor modelo durante as épocas;
      
  # Montando os datasets de Treino X e Y (começará a partir dar 4ª round e terá médias em vez de valores base, com as 50 primeira linhas depois das 30 iniciais):
  data_x_train = createX_Train(round_game, x_train_dataset, menor)  #|--------------------------------> Coleta as médias dos times Mandante e Visitante de cada partida e gera o dataset de treino X;
  data_y_train = createY_Train(round_game, menor)        #|--------------------------------> Coleta os "T" valores a partir da 4ª round (linha 30) do DF de resultados;
  x_train = tf.convert_to_tensor(data_x_train, dtype = tf.int32)  #|-----------> Convertendo o dataframe de treino de Estatísticas em Tensor de treino X;
  y_train = tf.convert_to_tensor(data_y_train, dtype = tf.int32)  #|-----------> Convertendo o dataframe de treino de Resultados em Tensor de treino Y;
  
  # Montando os sets de text (x e y):
  data_x_test = createX_Test(round_game, menor, roundGames, roundGames02)
  data_y_test = createY_Test(round_game, menor)
  x_test = tf.convert_to_tensor(data_x_test, dtype = tf.float32)  # |------------> Convertendo o dataframe de teste de Estatísticas em Tensor de teste X;
  y_test = tf.convert_to_tensor(data_y_test, dtype = tf.float32)  # |------------> Convertendo o dataframe de teste de Resultados em Tensor de teste Y;

  # 1.1 Treinando o modelo:
  lr_sched = step_decay_schedule(initial_lr=0.001, decay_factor=0.25, step_size=10)
    # 1.2 Parada Antecipada controlada:
  es = EarlyStopping(monitor='val_loss', mode='min', verbose = 0, patience = 17)
  history = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 50, callbacks =[es,mc], verbose = 0)

  # Predições:
  #  - Gerando predições para 10 jogos da round:
  dataPredict = createBasePredict(round_game,menor) # |-----------------------------> DF com o Resultado, OddH e OddLayH de cada jogo da round; (Atualizado em: 25_02_2021)
  predictions = model.predict(x_test)          # |------------------
  
  # Cálculo do Lucro/Prejuízo e dos Erros e Acertos da Rodada:
  acertosDaround = 0 
  errosDaround = 0
  LPround = 0

  for i in range(10):
    prob = predictions[i]               # recebe a probabilidade de vitoria ou não;
    
    if prob > 0.5:
      0        # oddH - odd de vitoria do mandante;
      cK = ((prob * oddH) - 1) / (oddH - 1)  # Criterio Kelly
      pA = np.round(porcentagemAposta,4)

    if pA <= 0:
      "não aposte"

     
       apostaBack = abs(montante) * porcentagemAposta
      
      if dataPredict.iloc[i][2] == 1:
        acertosDaround += 1
        montante += apostaBack * (oddH - 1)
        ganhos.append(apostaBack * (oddH - 1))
      else:
        errosDaround += 1
        montante -= apostaBack
        perdas.append(apostaBack)

    else:
      oddLayH = dataPredict.iloc[i][4]                        # oddLayH - odd de não vitoria do mandante;
      cK = ((1 - prob) * oddLayH) - 1 / (oddLayH - 1) # |------------> Verificar se isto está certo!!
      pA = abs(np.round(porcentagemAposta,4))

      if pA <= 0:
        "não aposte"

      "menos de 1000 ao fim da rodada, complica"
      
      apostaLay = montante*porcentagemAposta
      
      if dataPredict.iloc[i][2] == 0:
        acertosDaround += 1
        montante += apostaLay * (oddLayH - 1)
        ganhos.append(apostaBack * (oddLayH - 1))
      else:
        errosDaround += 1
        montante -= apostaLay
        perdas.append(apostaLay)

  
  acuraciaround = (acertosDaround/(acertosDaround+errosDaround))*100  # Calculo da acurácia da rodada;
  
  # Incrementado variáveis iteradoras:
  somaAcuracias += acuraciaround
  errosTotal += errosDaround
  acertosTotal += acertosDaround
  round_game += 1 
  
# RESULTADOS:
print("Seu montante final é de: ", montante)
print("Acurácia Média: %.2f %%" % (somaAcuracias/26))
#print("Taxa de erro: %.2f %%" % taxa_de_erros)

#sound_file = 'TF022.WAV'
#Audio(sound_file, autoplay=True)

In [ ]:

ganhosSuperiores = []

for ganho in ganhos:
  if ganho >= 100.00:
    ganhosSuperiores.append(ganho)

print("A quantidade de lucros superiores a R$ 100,00 foi de %d/%d. \n" % (len(ganhosSuperiores),acertosTotal))
print("Realizou %d empréstimos de R$ 100,00. Ou seja, gerou uma dívida de R$ %.2f." % (contadorEmprestimos,100*contadorEmprestimos))


temp = np.array(ganhos)
wins = temp.sum()

print("Acertou %d partidas. \n" % acertosTotal)
print("Ganhou R$ %.2f. \n"% wins)

temp = np.array(perdas) 

losses = temp.sum()


diff = wins - losses

print("Errou %d partidas. \n" % errosTotal)
print("Perdeu R$ %.2f reais. \n" % losses)
print("Saldo final: R$ %.2f. \n " % diff)

A quantidade de lucros superiores a R$ 100,00 foi de 1/152. 

Realizou 0 empréstimos de R$ 100,00. Ou seja, gerou uma dívida de R$ 0.00.
Acertou 152 partidas. 

Ganhou R$ 312.21. 

Errou 108 partidas. 

Perdeu R$ 639.18 reais. 

Saldo final: R$ -326.96. 
 
